# 01-Introduction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

#nota: esta bueno que utilicemos el 'ID' porque de esa manera podemos estar seguros de a que casa corresponde cada dato

In [3]:
X_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
X_full.shape

(1460, 80)

In [5]:
X_test_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


El dataser X_test NO tiene la columna `SalePrice` porque esa es justamente la variable que queresmo predecir con nuestro modelo

In [6]:
X_test_full.shape

(1459, 79)

### Obtain target and predictors

In [7]:
y = X_full.SalePrice

In [8]:
y

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64

In [9]:
# definimos cuales van a ser las columnas importantes para la prediccion de precios
# normalmente esta etapa puede requerir bastante investigacion, analisis y visualizacion de datos

features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

In [10]:
# X son los datos que vamos a usar para ENTRENAR el modelo y hacer la VALIDACION
X = X_full[features].copy()

# X-test lo vamos a usar para hacer el TEST
X_test = X_test_full[features].copy()

In [11]:
X.shape, X_test.shape

((1460, 7), (1459, 7))

In [12]:
# dividimos los datasets X (features) e y (valores verdders) en Training y Validation
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [13]:
X_train.shape, y_train.shape

((1168, 7), (1168,))

In [14]:
X_valid.shape, y_valid.shape

((292, 7), (292,))

### Definicion de los parametros del modelo
Vamos a utilizar un algoritmo `RanndomForestRegresor` y vamos a probar 5 modelos diferentes con variaciones

In [15]:
from sklearn.ensemble import RandomForestRegressor

model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100,criterion="absolute_error", random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

Para elegir cual de los 5 modelos es el mejor vamos a definir una funcion que nos permita valuar el error absoluto medio en cada uno de los medelos

In [16]:
from sklearn.metrics import mean_absolute_error

#definimos la funcion score_model
def score_model(model, X_t = X_train, y_t = y_train, X_v = X_valid, y_v = y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print(f'Model_{i+1} MAE:{mae.round(1)}')

Model_1 MAE:24015.5
Model_2 MAE:23741.0
Model_3 MAE:23528.8
Model_4 MAE:23996.7
Model_5 MAE:23706.7


Del resultado anterior podemos concluir que el mejor modelo, entre los que probamos, es el **model_3**. <br>
Por lo tanto nos vamos a quedar con ese modelo para seguir con este proyeco.

In [17]:
# 1. Definimos el modelo
my_model = RandomForestRegressor(n_estimators=100,criterion="absolute_error", random_state=0)
print('1. Model was defined\n')

# 2. Entrenamos el modelo
# notar que ahora podemos juntar los datos que antes habiamos separado para entrenamiento y validacion
my_model.fit(X, y)
print('2. Model was trained\n')

# 3. Prediccion.
preds_test = my_model.predict(X_test)
print('3. Predictions were made\n')

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission-01-introduction.csv', index=False)

print('4. File has been saved as "submission-01-introduction.csv"')

1. Model was defined

2. Model was trained

3. Predictions were made

4. File has been saved as "submission-01-introduction.csv"


In [18]:
output.shape

(1459, 2)

In [19]:
output.head()

,Id,SalePrice
0,1461,119433.08
1,1462,158367.50
2,1463,185351.21
3,1464,178343.12
4,1465,192898.29
